In [54]:

text=""
for i in range (1,7):
    filename='Harry_Potter_Text/'+'Book'+str(i)+'.txt'
    with open(filename) as f:
        book = f.read()
        text = text + book.lower()
tokens=text.split()
print(len(tokens))


957058


In [53]:
tokens

['/',
 'the',
 'boy',
 'who',
 'lived',
 'mr.',
 'and',
 'mrs.',
 'dursley,',
 'of',
 'number',
 'four,',
 'privet',
 'drive,',
 'were',
 'proud',
 'to',
 'say',
 'that',
 'they',
 'were',
 'perfectly',
 'normal,',
 'thank',
 'you',
 'very',
 'much.',
 'they',
 'were',
 'the',
 'last',
 'people',
 'you’d',
 'expect',
 'to',
 'be',
 'involved',
 'in',
 'anything',
 'strange',
 'or',
 'mysterious,',
 'because',
 'they',
 'just',
 'didn’t',
 'hold',
 'with',
 'such',
 'nonsense.',
 'mr.',
 'dursley',
 'was',
 'the',
 'director',
 'of',
 'a',
 'firm',
 'called',
 'grunnings,',
 'which',
 'made',
 'drills.',
 'he',
 'was',
 'a',
 'big,',
 'beefy',
 'man',
 'with',
 'hardly',
 'any',
 'neck,',
 'although',
 'he',
 'did',
 'have',
 'a',
 'very',
 'large',
 'mustache.',
 'mrs.',
 'dursley',
 'was',
 'thin',
 'and',
 'blonde',
 'and',
 'had',
 'nearly',
 'twice',
 'the',
 'usual',
 'amount',
 'of',
 'neck,',
 'which',
 'came',
 'in',
 'very',
 'useful',
 'as',
 'she',
 'spent',
 'so',
 'much',


In [57]:
vocab=list(set(tokens))

In [59]:
print(len(vocab))

49645


In [73]:
ngrams=[{},{},{},{},{}]
for n in range (1,6):
    for i in range(len(tokens)):
        if i+n>len(tokens):
            break
        lis=[]
        for j in range(i,i+n):
            lis.append(tokens[j])
        lis=tuple(lis)
        if lis not in ngrams[n-1]:
            ngrams[n-1][lis] = 1
        else : 
            ngrams[n-1][lis] += 1
    ngrams[n-1]=dict(sorted(ngrams[n-1].items(), key=lambda item: -item[1]))
    print(n,len(ngrams[n-1]))




1 49645
2 369022
3 711287
4 863950
5 910407
